In [1]:
import pandas as pd
import json
import time
import subprocess
import torch
from model_prompt import KGPrompt
from collections import defaultdict
from dataset_dbpedia import DBpedia

import sys
sys.path.append('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG')

from pre_rec_dataset_dbpedia import Dialogue_DBpedia
from dataset_dbpedia import DBpedia

In [2]:
# with open('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/dialogues/test_data copy.txt', 'r') as file:
#     for i,line in enumerate(file):
#         print("Processing line ", i)
#         line = line.strip()
#         with open('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/input/current_line.txt', 'w') as current_file:
#            current_file.write(line)

#         # Run the graphrag command and wait for it to complete
#         process = subprocess.run(['graphrag', 'index', '--root', '/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/'], 
#                               capture_output=True,  # Capture the output
#                               text=True)  # Convert output to string
        
#         if process.returncode == 0:
#             print("Command completed successfully")
            

In [3]:
# Read the generated parquet files
entities_df = pd.read_parquet('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/output/create_final_entities.parquet')
relationships_df = pd.read_parquet('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/output/create_final_relationships.parquet')

print("Found entities:", len(entities_df))
print("First 5 entities", entities_df['title'])
print("Found relationships:", len(relationships_df))

Found entities: 16
First 5 entities 0      A NIGHTMARE ON ELM STREET (1984)
1                HAPPY DEATH DAY (2017)
2     THE LAST HOUSE ON THE LEFT (1972)
3                  A QUIET PLACE (2018)
4                      ANNABELLE (2014)
5                     THE FOREST (2016)
6                  HOT TUB TIME MACHINE
7                 SUPER TROOPERS (2001)
8                 IDENTITY THIEF (2013)
9                       THE HEAT (2013)
10                               HORROR
11                               COMEDY
12                       JOHN KRASINSKI
13                        JASON BATEMAN
14                     MELISSA MCCARTHY
15                       FREDDY KRUEGER
Name: title, dtype: object
Found relationships: 15


In [4]:
# --------- Assign Relationship Label --------------

# Prepare mappings for entities
entity_id_map = dict(zip(entities_df['title'], entities_df['human_readable_id']))
entity_type_map = dict(zip(entities_df['title'], entities_df['type']))

# Create Relationships DataFrame with initial mappings
relationships_clean_df = relationships_df.assign(
    entity_id_source=relationships_df['source'].map(entity_id_map),
    entity_type_source=relationships_df['source'].map(entity_type_map),
    entity_id_target=relationships_df['target'].map(entity_id_map),
    entity_type_target=relationships_df['target'].map(entity_type_map)
)

# Define relationship type mapping with IDs
relationship_type_mapping = {
    ('MOVIE', 'ACTOR'): ('features', 0),
    ('ACTOR', 'MOVIE'): ('acted_in', 1),
    ('MOVIE', 'GENRE'): ('belongs_to_genre', 2),
    ('GENRE', 'MOVIE'): ('categorizes', 3),
    ('MOVIE', 'DIRECTOR'): ('directed_by', 4),
    ('DIRECTOR', 'MOVIE'): ('directed', 5),
    ('MOVIE', 'CHARACTER'): ('has_character', 6),
    ('CHARACTER', 'MOVIE'): ('featured_in', 7),
    ('ACTOR', 'CHARACTER'): ('portrays', 8),
    ('CHARACTER', 'ACTOR'): ('portrayed_by', 9),
    ('GENRE', 'DIRECTOR'): ('prefers_to_direct', 10),
    ('DIRECTOR', 'GENRE'): ('has_preference_for', 11),
    ('CHARACTER', 'DIRECTOR'): ('created_by', 12),
    ('DIRECTOR', 'CHARACTER'): ('conceived', 13),
    ('MOVIE', 'MOVIE'): ('similar', 14)
}

# Reverse the relationships and add them to the original DataFrame
relationships_reversed_df = relationships_clean_df.rename(columns={
    'entity_id_source': 'entity_id_target',
    'entity_name_source': 'entity_name_target',
    'entity_type_source': 'entity_type_target',
    'entity_id_target': 'entity_id_source',
    'entity_name_target': 'entity_name_source',
    'entity_type_target': 'entity_type_source'
}).copy()

# Combine the original and reversed relationships
relationships_combined_df = pd.concat([relationships_clean_df, relationships_reversed_df], ignore_index=True)

# Assign relationship types and IDs based on source and target entity types
relationships_combined_df[['relationship_type', 'relationship_type_id']] = relationships_combined_df.apply(
    lambda row: pd.Series(relationship_type_mapping.get(
        (row['entity_type_source'], row['entity_type_target']), ('unknown', -1)
    )),
    axis=1
)

kg_df = relationships_combined_df[['entity_id_source', 'source', 'entity_type_source',  'entity_id_target', 'target', 'entity_type_target', 'relationship_type', 'relationship_type_id']]


In [ ]:
# --------- Create file dialogue_subkg.json --------------

# Group data by source entity
json_structure = defaultdict(list)

# Replace 'kg_df' with the DataFrame containing the KG relationships
for _, row in kg_df.iterrows():
    source_id = row['entity_id_source']
    target_id = row['entity_id_target']
    relationship_id = row['relationship_type_id']
    if relationship_id != -1:  # Only include valid relationships
        json_structure[source_id].append([relationship_id, target_id])

# Convert the defaultdict to a regular dictionary
json_output = {str(source): [[rel_id, target] for rel_id, target in targets]
               for source, targets in json_structure.items()}

# Save the compact JSON structure to a file
compact_json_file_path = "/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/dialogue_subkg.json"
with open(compact_json_file_path, "w") as json_file:
    json.dump(json_output, json_file, separators=(',', ':'))

In [6]:
# --------- Create file dialogue_entity2id.json --------------

# Create a mapping of entity: id in the required format
entity_id_mapping = {
    f"{row['title']}" : row['human_readable_id']
    for _, row in entities_df.iterrows()
}

# Save the mapping as a JSON file
entity_json_file_path = "/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/dialogue_entity2id.json"
with open(entity_json_file_path, "w") as json_file:
    json.dump(entity_id_mapping, json_file, separators=(',', ':'))

In [7]:
# --------- Create file dialogue_item_ids.json --------------
item_set = set()

# store entity ids in item_set
item_set = set(entities_df['human_readable_id'].tolist())

with open('/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/dialogue_item_ids.json', 'w', encoding='utf-8') as f:
    json.dump(list(item_set), f, ensure_ascii=False)

In [8]:
# --------- GET DIALOGUE EMBEDDINGS VIA RGCN --------------

def get_dialogue_embeddings():
    """
    Get RGCN embeddings for a single dialogue's KG
    
    Args:
        dialogue_path: Path to folder containing the dialogue's KG files
                      (dbpedia_subkg.json, entity2id.json, relation2id.json, item_ids.json)
    """
    # Initialize DBpedia with dialogue data
    kg = Dialogue_DBpedia(debug=True).get_entity_kg_info()
    
    # Initialize KGPrompt (which contains the RGCN)
    prompt_encoder = KGPrompt(
        hidden_size=768,              # GPT2 hidden size
        token_hidden_size=768,        # Same as hidden_size
        n_head=12,                    # Number of attention heads
        n_layer=12,                   # Number of layers
        n_block=2,                    # Number of blocks
        n_entity=kg['num_entities'],  # From your KG
        num_relations=kg['num_relations'],  # From your KG
        num_bases=8,                  # Number of bases for relation weights
        edge_index=kg['edge_index'],  # From your KG
        edge_type=kg['edge_type']     # From your KG
    )
    
    # Get embeddings
    with torch.no_grad():
        entity_embeddings = prompt_encoder.get_entity_embeds()
    
    return entity_embeddings, prompt_encoder, kg

# Example usage
if __name__ == "__main__":
    
    # Get embeddings
    embeddings, encoder, kg_info = get_dialogue_embeddings()
    
    print(f"Number of entities: {kg_info['num_entities']}")
    print(f"Number of relations: {kg_info['num_relations']}")
    print(f"Embedding shape: {embeddings.shape}")
    
    # Save the embeddings if needed
    torch.save(embeddings, f"/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/entity_embeddings.pt")

2025-02-22 15:00:27.277 | DEBUG    | pre_rec_dataset_dbpedia:_process_entity_kg:43 - #edge: 16, #relation: 15, #entity: 17, #item: 16


node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([17, 384])
edge_index shape: torch.Size([2, 16])
edge_type shape: torch.Size([16])
Number of entities: 17
Number of relations: 15
Embedding shape: torch.Size([17, 768])


In [10]:
# --------- GET KG EMBEDDINGS VIA RGCN --------------

def get_kg_embeddings():
    """
    Get RGCN embeddings for a single dialogue's KG
    
    Args:
        dialogue_path: Path to folder containing the dialogue's KG files
                      (dbpedia_subkg.json, entity2id.json, relation2id.json, item_ids.json)
    """
    # Initialize DBpedia with dialogue data
    kg = DBpedia(debug=True).get_entity_kg_info()
    
    # Initialize KGPrompt (which contains the RGCN)
    prompt_encoder = KGPrompt(
        hidden_size=768,              # GPT2 hidden size
        token_hidden_size=768,        # Same as hidden_size
        n_head=12,                    # Number of attention heads
        n_layer=12,                   # Number of layers
        n_block=2,                    # Number of blocks
        n_entity=kg['num_entities'],  # From your KG
        num_relations=kg['num_relations'],  # From your KG
        num_bases=8,                  # Number of bases for relation weights
        edge_index=kg['edge_index'],  # From your KG
        edge_type=kg['edge_type']     # From your KG
    )
    
    # Get embeddings
    with torch.no_grad():
        entity_embeddings = prompt_encoder.get_entity_embeds()
    
    return entity_embeddings, prompt_encoder, kg

# Example usage
if __name__ == "__main__":
    
    # Get embeddings
    embeddings, encoder, kg_info = get_kg_embeddings()
    
    print(f"Number of entities: {kg_info['num_entities']}")
    print(f"Number of relations: {kg_info['num_relations']}")
    print(f"Embedding shape: {embeddings.shape}")
    
    # Save the embeddings if needed
    torch.save(embeddings, f"/home/Nema/UniCRS_GraphRAG/Recommendation_GraphRAG/entity_embeddings.pt")


2025-02-22 15:00:46.548 | DEBUG    | dataset_dbpedia:_process_entity_kg:44 - #edge: 45180, #relation: 15, #entity: 11861, #item: 4776


node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([11861, 384])
edge_index shape: torch.Size([2, 45180])
edge_type shape: torch.Size([45180])
Number of entities: 11861
Number of relations: 15
Embedding shape: torch.Size([11861, 768])


In [ ]:
def get_both_embeddings():
    """Get embeddings for both KGs using the same RGCN"""
    
    # First get the main KG RGCN
    print("Initializing main KG...")
    main_kg = DBpedia(debug=True).get_entity_kg_info()
    
    # Initialize RGCN with main KG
    prompt_encoder = KGPrompt(
        hidden_size=768,
        token_hidden_size=768,
        n_head=12,
        n_layer=12,
        n_block=2,
        n_entity=main_kg['num_entities'],
        num_relations=main_kg['num_relations'],
        num_bases=8,
        edge_index=main_kg['edge_index'],
        edge_type=main_kg['edge_type']
    )
    
    # Get main KG embeddings
    with torch.no_grad():
        main_embeddings = prompt_encoder.get_entity_embeds()
    
    print("Getting dialogue KG...")
    dialogue_kg = Dialogue_DBpedia(debug=True).get_entity_kg_info()
    
    # Store original graph info
    orig_edge_index = prompt_encoder.edge_index
    orig_edge_type = prompt_encoder.edge_type
    
    # Temporarily replace with dialogue subgraph
    prompt_encoder.edge_index.data = dialogue_kg['edge_index']
    prompt_encoder.edge_type.data = dialogue_kg['edge_type']
    
    # Get dialogue embeddings using same RGCN
    with torch.no_grad():
        dialogue_embeddings = prompt_encoder.get_entity_embeds()
    
    # Restore original graph
    prompt_encoder.edge_index.data = orig_edge_index
    prompt_encoder.edge_type.data = orig_edge_type
    
    return dialogue_embeddings, main_embeddings, dialogue_kg, main_kg

In [18]:
import torch
import torch.nn.functional as F
import networkx as nx
from typing import Dict, List, Set, Tuple

def extract_relevant_subgraph(dialogue_embeddings: torch.Tensor, 
                            kg_embeddings: torch.Tensor,
                            main_kg: Dict,
                            dialogue_entities: List[str],
                            similarity_threshold: float = 0.7,
                            top_k: int = 5,
                            hop_size: int = 2) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Extract a subgraph from main KG based on dialogue entity similarities
    
    Args:
        dialogue_embeddings: Embeddings from dialogue KG
        kg_embeddings: Embeddings from main KG
        main_kg: Main KG info containing edge_index and edge_type
        dialogue_entities: List of dialogue entity titles
        similarity_threshold: Minimum similarity score to consider
        top_k: Number of similar entities to consider per dialogue entity
        hop_size: Number of hops to expand from similar entities
    
    Returns:
        Tuple of (edge_index, edge_type) for the extracted subgraph
    """
    # Normalize embeddings
    dialogue_embeddings_norm = F.normalize(dialogue_embeddings, p=2, dim=1)
    kg_embeddings_norm = F.normalize(kg_embeddings, p=2, dim=1)
    
    # Find similar entities
    similar_entity_ids = set()
    for dialogue_emb in dialogue_embeddings_norm:
        # Compute similarities with all KG entities
        similarities = F.cosine_similarity(dialogue_emb.unsqueeze(0), kg_embeddings_norm)
        
        # Get top-k similar entities
        scores, indices = torch.topk(similarities, k=min(top_k, len(kg_embeddings_norm)))
        
        # Add entities above threshold
        for score, idx in zip(scores, indices):
            if score >= similarity_threshold:
                similar_entity_ids.add(idx.item())
    
    print(f"Found {len(similar_entity_ids)} similar entities")
    
    # Convert main KG to networkx for easier neighbor extraction
    edge_index = main_kg['edge_index']
    edge_type = main_kg['edge_type']
    
    G = nx.Graph()
    for i in range(edge_index.size(1)):
        src = edge_index[0, i].item()
        dst = edge_index[1, i].item()
        G.add_edge(src, dst)
    
    # Expand subgraph by hop_size
    expanded_entities = similar_entity_ids.copy()
    for _ in range(hop_size):
        entities_to_add = set()
        for entity_id in expanded_entities:
            if entity_id in G:
                neighbors = set(G.neighbors(entity_id))
                entities_to_add.update(neighbors)
        expanded_entities.update(entities_to_add)
    
    print(f"Expanded to {len(expanded_entities)} entities after {hop_size} hops")
    
    # Extract edges for subgraph
    subgraph_edges = []
    subgraph_types = []
    
    for i in range(edge_index.size(1)):
        src = edge_index[0, i].item()
        dst = edge_index[1, i].item()
        if src in expanded_entities and dst in expanded_entities:
            subgraph_edges.append([src, dst])
            subgraph_types.append(edge_type[i].item())
    
    # Convert to tensors
    subgraph_edge_index = torch.tensor(subgraph_edges, dtype=torch.long).t()
    subgraph_edge_type = torch.tensor(subgraph_types, dtype=torch.long)
    
    print(f"Extracted subgraph has {len(subgraph_types)} edges")
    
    return subgraph_edge_index, subgraph_edge_type

# Get both embeddings using same RGCN
dialogue_embeddings, kg_embeddings, dialogue_kg, main_kg = get_both_embeddings()

# Get dialogue entity titles
entity_titles = entities_df['title'].tolist()

# Extract subgraph
subgraph_edge_index, subgraph_edge_type = extract_relevant_subgraph(
    dialogue_embeddings,
    kg_embeddings,
    main_kg,
    entity_titles,
    similarity_threshold=0.5,
    top_k=5,
    hop_size=1
)

# Print some statistics about the extracted subgraph
print("\nSubgraph Statistics:")
print(f"Number of edges: {subgraph_edge_type.size(0)}")
print(f"Number of unique entities: {len(torch.unique(subgraph_edge_index))}")
print(f"Number of unique relation types: {len(torch.unique(subgraph_edge_type))}")

# Optionally, print some example edges
print("\nExample edges in subgraph:")
for i in range(min(5, subgraph_edge_type.size(0))):
    src = subgraph_edge_index[0, i].item()
    dst = subgraph_edge_index[1, i].item()
    rel = subgraph_edge_type[i].item()
    print(f"Entity {src} -> Relation {rel} -> Entity {dst}")

2025-02-22 15:44:38.390 | DEBUG    | dataset_dbpedia:_process_entity_kg:44 - #edge: 45180, #relation: 15, #entity: 11861, #item: 4776


Initializing main KG...
node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([11861, 384])
edge_index shape: torch.Size([2, 45180])
edge_type shape: torch.Size([45180])


2025-02-22 15:44:39.750 | DEBUG    | pre_rec_dataset_dbpedia:_process_entity_kg:43 - #edge: 16, #relation: 15, #entity: 17, #item: 16


Getting dialogue KG...
node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([11861, 384])
edge_index shape: torch.Size([2, 16])
edge_type shape: torch.Size([16])
Found 11825 similar entities
Expanded to 11861 entities after 1 hops
Extracted subgraph has 45180 edges

Subgraph Statistics:
Number of edges: 45180
Number of unique entities: 6181
Number of unique relation types: 13

Example edges in subgraph:
Entity 171 -> Relation 5 -> Entity 7435
Entity 10402 -> Relation 3 -> Entity 2561
Entity 131 -> Relation 1 -> Entity 303
Entity 1469 -> Relation 1 -> Entity 2437
Entity 754 -> Relation 2 -> Entity 9008


In [16]:
import torch
import torch.nn.functional as F
from typing import Dict, List, Tuple
import pandas as pd
import numpy as np

def get_both_embeddings():
    """Get embeddings for both KGs using the same RGCN"""
    
    # First get the main KG RGCN
    print("Initializing main KG...")
    main_kg = DBpedia(debug=True).get_entity_kg_info()
    
    # Initialize RGCN with main KG
    prompt_encoder = KGPrompt(
        hidden_size=768,
        token_hidden_size=768,
        n_head=12,
        n_layer=12,
        n_block=2,
        n_entity=main_kg['num_entities'],
        num_relations=main_kg['num_relations'],
        num_bases=8,
        edge_index=main_kg['edge_index'],
        edge_type=main_kg['edge_type']
    )
    
    # Get main KG embeddings
    with torch.no_grad():
        main_embeddings = prompt_encoder.get_entity_embeds()
    
    print("Getting dialogue KG...")
    dialogue_kg = Dialogue_DBpedia(debug=True).get_entity_kg_info()
    
    # Store original graph info
    orig_edge_index = prompt_encoder.edge_index
    orig_edge_type = prompt_encoder.edge_type
    
    # Temporarily replace with dialogue subgraph
    prompt_encoder.edge_index.data = dialogue_kg['edge_index']
    prompt_encoder.edge_type.data = dialogue_kg['edge_type']
    
    # Get dialogue embeddings using same RGCN
    with torch.no_grad():
        dialogue_embeddings = prompt_encoder.get_entity_embeds()
    
    # Restore original graph
    prompt_encoder.edge_index.data = orig_edge_index
    prompt_encoder.edge_type.data = orig_edge_type
    
    return dialogue_embeddings, main_embeddings, dialogue_kg, main_kg

def match_entities(dialogue_embeddings, kg_embeddings, entity_titles, 
                  similarity_threshold=0.5, top_k=3):
    """Match entities based on embedding similarity"""
    
    dialogue_embeddings_norm = F.normalize(dialogue_embeddings, p=2, dim=1)
    kg_embeddings_norm = F.normalize(kg_embeddings, p=2, dim=1)
    
    matches = []
    for i, dialogue_emb in enumerate(dialogue_embeddings_norm):
        # Get entity title
        entity_title = entity_titles[i] if i < len(entity_titles) else f"Entity_{i}"
        
        # Compute similarities
        similarities = F.cosine_similarity(dialogue_emb.unsqueeze(0), kg_embeddings_norm)
        
        # Get top matches
        scores, indices = torch.topk(similarities, k=min(top_k, len(kg_embeddings_norm)))
        
        entity_matches = []
        for score, idx in zip(scores, indices):
            score = score.item()
            if score >= similarity_threshold:
                entity_matches.append({
                    'similarity_score': score,
                    'kg_entity_idx': idx.item()
                })
        
        matches.append({
            'dialogue_entity_idx': i,
            'entity_title': entity_title,
            'matches': entity_matches
        })
        
    return matches

# Get embeddings using same RGCN
dialogue_embeddings, kg_embeddings, dialogue_kg, main_kg = get_both_embeddings()

# Get entity titles
entity_titles = entities_df['title'].tolist()

# Match entities
matches = match_entities(dialogue_embeddings, kg_embeddings, entity_titles)

# Print results
print("\nEntity Matching Results:")
for match in matches:
    print(f"\nDialogue Entity: {match['entity_title']}")
    if match['matches']:
        print("Top matches:")
        for m in match['matches']:
            kg_idx = m['kg_entity_idx']
            score = m['similarity_score']
            print(f"  - KG Entity {kg_idx} (similarity: {score:.3f})")
    else:
        print("No matches found above threshold")

2025-02-22 15:10:34.522 | DEBUG    | dataset_dbpedia:_process_entity_kg:44 - #edge: 45180, #relation: 15, #entity: 11861, #item: 4776


Initializing main KG...
node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([11861, 384])
edge_index shape: torch.Size([2, 45180])
edge_type shape: torch.Size([45180])


2025-02-22 15:10:35.771 | DEBUG    | pre_rec_dataset_dbpedia:_process_entity_kg:43 - #edge: 16, #relation: 15, #entity: 17, #item: 16


Getting dialogue KG...
node_embeds device: cpu
edge_index device: cpu
edge_type device: cpu
node_embeds shape: torch.Size([11861, 384])
edge_index shape: torch.Size([2, 16])
edge_type shape: torch.Size([16])

Entity Matching Results:

Dialogue Entity: A NIGHTMARE ON ELM STREET (1984)
Top matches:
  - KG Entity 6216 (similarity: 0.953)
  - KG Entity 0 (similarity: 0.951)
  - KG Entity 9384 (similarity: 0.950)

Dialogue Entity: HAPPY DEATH DAY (2017)
Top matches:
  - KG Entity 2511 (similarity: 0.950)
  - KG Entity 11670 (similarity: 0.948)
  - KG Entity 8252 (similarity: 0.948)

Dialogue Entity: THE LAST HOUSE ON THE LEFT (1972)
Top matches:
  - KG Entity 2 (similarity: 0.992)
  - KG Entity 10721 (similarity: 0.950)
  - KG Entity 4350 (similarity: 0.949)

Dialogue Entity: A QUIET PLACE (2018)
Top matches:
  - KG Entity 3 (similarity: 0.969)
  - KG Entity 10718 (similarity: 0.931)
  - KG Entity 3873 (similarity: 0.931)

Dialogue Entity: ANNABELLE (2014)
Top matches:
  - KG Entity 10442 (